# <b>A3.1 SVM y Multiple Testing</b>

In [2]:
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import numpy as np
from scipy.stats import f_oneway
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

#### 1. Se carga el conjunto de datos con todos los genes para poder trabajar con ellos. Después se revisa que no haya huecos en los huecos en cada columna, y se imprime la confirmación en pantalla como un valor booleano.

In [3]:
# Cargar el archivo CSV
df = pd.read_csv('A3.1 Khan.csv')
# Mostrar dimensiones y valores faltantes. La funcion .isnull() regresa el mismo DataFrame, pero en vez de los datos, regresa valores Booleanos, True si estan faltantes, False si el valor esta presente.
# .any() returns True if at least one value in the specified axis is True. applied to columns, by default axis=0. Now you get a series saying which columns have missing values.
# To grab a particular column, one has to
# .any().any() 

print("¿Hay huecos?: ", df.isnull().any().any())

¿Hay huecos?:  False


#### A través de la lógica que los métodos de DataFrame posee, podemos proceder seguramente con las instrucciones siguientes al saber que no hay huecos.

#### Se calculan y ordenan las diferencias absolutas correctamente, y se muestran los 10 genes principales.

In [4]:
# Suponemos que la última columna es la variable de salida con valores del 1 al 4
# Asegúrate de renombrarla si no está claramente nombrada
target_col = df.columns[-1]
print("\nClases únicas en la columna objetivo:", df[target_col].unique())

# Separar clases 2 y 4
class_2 = df[df[target_col] == 2].drop(columns=target_col)
class_4 = df[df[target_col] == 4].drop(columns=target_col)

# Calcular diferencia de medias
diff_means = (class_2.mean() - class_4.mean()).abs().sort_values(ascending=False)

# Mostrar los 10 genes con mayor diferencia
print("\nTop 10 genes con mayor diferencia de medias entre clase 2 y 4:")
print(diff_means.head(10))


Clases únicas en la columna objetivo: [2 4 3 1]

Top 10 genes con mayor diferencia de medias entre clase 2 y 4:
X187     3.323151
X509     2.906537
X2046    2.424515
X2050    2.401783
X129     2.165185
X1645    2.065460
X1319    2.045941
X1955    2.037340
X1003    2.011337
X246     1.837830
dtype: float64


#### Los genes listados (por ejemplo `X187`, `X509`, `X2046`) presentan mayores diferencias de expresión, lo cual sugiere que podrían ser biomarcadores entre los tipos de cáncer 2 y 4.

#### 2. Cálculo del estadístico t y corrección por múltiples pruebas. Se realiza t-test (Welch) por gen, y se aplican Bonferroni, Holm y Benjamini-Hochberg (FDR) con alpha=0.05.

In [7]:

# Calcular t-test para cada gen
t_stats = []
p_values = []

for gene in class_2.columns:
    t, p = ttest_ind(class_2[gene], class_4[gene], equal_var=False)
    t_stats.append(t)
    p_values.append(p)

# Correcciones por múltiples pruebas
methods = ['bonferroni', 'holm', 'fdr_bh']
corrections = {}

for method in methods:
    reject, p_corr, _, _ = multipletests(p_values, alpha=0.05, method=method)
    corrections[method] = {
        'genes_significativos': df.columns[:-1][reject],
        'pvalores_corregidos': p_corr[reject]
    }

# Mostrar resultados
for method, res in corrections.items():
    print(f"\nMétodo: {method}")
    print("Genes con diferencias significativas:", list(res['genes_significativos']))
    print(len(res['genes_significativos']))



Método: bonferroni
Genes con diferencias significativas: ['X2', 'X36', 'X67', 'X129', 'X174', 'X187', 'X188', 'X229', 'X246', 'X251', 'X338', 'X348', 'X368', 'X372', 'X373', 'X380', 'X430', 'X433', 'X509', 'X545', 'X554', 'X558', 'X566', 'X603', 'X655', 'X714', 'X762', 'X910', 'X951', 'X971', 'X1003', 'X1021', 'X1023', 'X1055', 'X1070', 'X1093', 'X1105', 'X1110', 'X1112', 'X1132', 'X1194', 'X1196', 'X1207', 'X1217', 'X1298', 'X1319', 'X1327', 'X1330', 'X1372', 'X1389', 'X1416', 'X1610', 'X1626', 'X1634', 'X1645', 'X1706', 'X1708', 'X1723', 'X1738', 'X1799', 'X1888', 'X1896', 'X1911', 'X1924', 'X1954', 'X1955', 'X1980', 'X2046', 'X2050', 'X2115', 'X2146', 'X2247']
72

Método: holm
Genes con diferencias significativas: ['X2', 'X36', 'X67', 'X129', 'X174', 'X187', 'X188', 'X229', 'X246', 'X251', 'X338', 'X348', 'X368', 'X372', 'X373', 'X380', 'X430', 'X433', 'X509', 'X545', 'X554', 'X558', 'X566', 'X603', 'X655', 'X714', 'X762', 'X910', 'X951', 'X971', 'X1003', 'X1021', 'X1023', 'X1055',

| Método               | # Genes significativos |
|----------------------|------------------------|
| bonferroni           | 72                     |
| holm                 | 72                     |
| Benjamini-Hochberg   | 296                    |

####   A través de esto podemos notar que a medida que el método es menos conservador, aumenta el número de genes significativos.

#### 3. Se utilizará ANOVA multiclas para las 4 clases y se corrige FDR con BH. El análisis de varianza identifica genes con expresión diferencial entre los cuatro tipos de cáncer.

In [9]:
# Estratificar por clase
grouped = [df[df[target_col] == c].drop(columns=target_col) for c in sorted(df[target_col].unique())]

# Realizar ANOVA para cada gen
p_values_anova = []
for gene in df.columns[:-1]:
    f_stat, p = f_oneway(*(group[gene] for group in grouped))
    p_values_anova.append(p)

# Corrección Benjamini-Hochberg para ANOVA
reject_anova, pvals_corr_anova, _, _ = multipletests(p_values_anova, alpha=0.05, method='fdr_bh')
genes_signif_anova = df.columns[:-1][reject_anova]

print("Genes significativamente diferentes entre las 4 clases (ANOVA + BH):")
print(list(genes_signif_anova))
print("Cantidad total: ", len(list(genes_signif_anova)))

Genes significativamente diferentes entre las 4 clases (ANOVA + BH):
['X1', 'X2', 'X3', 'X9', 'X12', 'X17', 'X21', 'X22', 'X27', 'X29', 'X32', 'X33', 'X36', 'X37', 'X45', 'X46', 'X50', 'X52', 'X54', 'X56', 'X60', 'X64', 'X65', 'X66', 'X67', 'X68', 'X70', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78', 'X80', 'X81', 'X84', 'X85', 'X88', 'X89', 'X90', 'X92', 'X93', 'X94', 'X95', 'X96', 'X97', 'X98', 'X99', 'X100', 'X107', 'X108', 'X116', 'X119', 'X121', 'X123', 'X124', 'X125', 'X127', 'X129', 'X130', 'X131', 'X132', 'X135', 'X139', 'X141', 'X142', 'X144', 'X145', 'X146', 'X147', 'X151', 'X152', 'X153', 'X154', 'X156', 'X159', 'X160', 'X165', 'X166', 'X167', 'X169', 'X171', 'X174', 'X175', 'X178', 'X182', 'X185', 'X186', 'X187', 'X188', 'X190', 'X191', 'X192', 'X194', 'X198', 'X200', 'X201', 'X204', 'X205', 'X207', 'X208', 'X212', 'X213', 'X214', 'X216', 'X217', 'X219', 'X220', 'X222', 'X224', 'X225', 'X229', 'X230', 'X231', 'X233', 'X234', 'X235', 'X236', 'X239', 'X240', 'X243', 'X244', 'X245'

#### 4. Entrenamiento de modelo SVM con kernel lineal. Se seleccionan los top 10 genes del paso 2 (reconociendo la fuga de datos) y se comparan kernels lineal, polinomial (grado 3) y RBF.

In [13]:
# Selección de características: top 10 genes con mayor diferencia de medias
top_genes = diff_means.head(10).index.tolist()

X = df[top_genes]
y = df[target_col]

# Separar datos
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

# Modelos SVM
kernels = {
    'lineal': SVC(kernel='linear'),
    'polinomial': SVC(kernel='poly', degree=3),
    'radial': SVC(kernel='rbf')
}

resultados = {}

for nombre, modelo in kernels.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    resultados[nombre] = classification_report(y_test, y_pred, output_dict=True)
    print(f"\nClasificación con kernel {nombre}:")
    print(classification_report(y_test, y_pred))


Clasificación con kernel lineal:
              precision    recall  f1-score   support

           1       0.50      0.50      0.50         2
           2       1.00      1.00      1.00         6
           3       0.75      0.75      0.75         4
           4       1.00      1.00      1.00         5

    accuracy                           0.88        17
   macro avg       0.81      0.81      0.81        17
weighted avg       0.88      0.88      0.88        17


Clasificación con kernel polinomial:
              precision    recall  f1-score   support

           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         6
           3       0.80      1.00      0.89         4
           4       1.00      1.00      1.00         5

    accuracy                           0.94        17
   macro avg       0.95      0.88      0.89        17
weighted avg       0.95      0.94      0.93        17


Clasificación con kernel radial:
              precision 

#### 5. Comparación de resultados de los tres modelos anteriores (lineal, polinomial y radial).

In [14]:
# Comparar precisión macro
scores = {
    k: round(v['macro avg']['f1-score'], 3)
    for k, v in resultados.items()
}

pd.DataFrame.from_dict(scores, orient='index', columns=['F1 macro']).sort_values(by='F1 macro', ascending=False)


,F1 macro
radial,1.000
polinomial,0.889
lineal,0.812


#### El kernel radial obtienen F1 macro = 1.00, mientras que el polinomial cae a 0.89 y el lineal a 0.81; por lo tanto, para este conjunto de características, un RBF es más adecuado. Los genes identificados como más diferenciales coinciden con aquellos que mejor separaron las clases en el SVM, y el kernel RBF resulta óptimo para este subset.